# OA Options / Other Normalised
Build two Silver tables from OA raw content using LLM section extraction:
- `silver_md.md_dta_oa_options_draft` - sections with well-populated option metadata
- `silver_md.md_dta_oa_other_draft` - sections with missing / incomplete section fields

**Input**: `bronze_md.md_oa_file_raw`, `silver_md.md_dta_operational_agreement_draft`

**Output**: `silver_md.md_dta_oa_options_draft`, `silver_md.md_dta_oa_other_draft`


In [ ]:
# Cell 0: Imports and UDF

import json
import requests
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField,
    StringType, ArrayType, IntegerType, BooleanType
)
from clinical_data_standards_framework.utils import save_with_audit, compute_definition_hash

print("=" * 80)
print("OA OPTIONS / OTHER NORMALISATION (Silver)")
print("=" * 80)

# OA definition hash UDF
@F.udf(StringType())
def oa_definition_hash_udf(
    operational_agreement_id: str,
    trial_id: str,
    data_stream: str,
    data_provider_name: str
) -> str:
    record = {
        "operational_agreement_id": operational_agreement_id or "",
        "trial_id": trial_id or "",
        "data_stream": data_stream or "",
        "data_provider_name": data_provider_name or "",
    }
    return compute_definition_hash(record, list(record.keys()))

print("✅ Framework loaded successfully")


In [ ]:
# Cell 1: Read config from setup notebook task

globals_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
)
services_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="services")
)

try:
    pipeline_config = json.loads(
        dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config")
    )
except Exception:
    pipeline_config = None

flow_name = dbutils.jobs.taskValues.get(taskKey="setup", key="flow_name")

created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name  = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

if not pipeline_config:
    raise ValueError(
        "pipeline_config task value is missing; "
        "ensure setup task ran with flow_name='operational_agreement_processor'"
    )

# Resolve OA pipeline config
docs_cfg   = pipeline_config["documents"]["operational_agreement"]
output_cfg = docs_cfg["output"]

# From globals
catalog       = globals_dict["catalog"]
bronze_schema = globals_dict["bronze_schema"]
silver_schema = globals_dict["silver_schema"]
gold_schema   = globals_dict["gold_schema"]

# From OA pipeline config
raw_table_name   = output_cfg["raw_table_name"]
pages_table_name = output_cfg.get("pages_table_name", None)

raw_table_full = f"{catalog}.{bronze_schema}.{raw_table_name}"
options_table  = f"{catalog}.{silver_schema}.md_dta_oa_options_draft"
other_table    = f"{catalog}.{silver_schema}.md_dta_oa_other_draft"

print("\nResolved config:")
print(f"  catalog          = {catalog}")
print(f"  bronze_schema    = {bronze_schema}")
print(f"  silver_schema    = {silver_schema}")
print(f"  raw_table_full   = {raw_table_full}")
print(f"  options_table    = {options_table}")
print(f"  other_table      = {other_table}")
print(f"  pages_table_name = {pages_table_name} (optional)")

spark.sql(f"USE CATALOG `{catalog}`")


In [ ]:
# Cell 2: Build a 'pages' DataFrame

# If a pages table is configured and exists, use it.
# Otherwise, fall back to treating each full document as a single "page".

if pages_table_name and spark.catalog.tableExists(f"{catalog}.{bronze_schema}.{pages_table_name}"):
    pages_table_full = f"{catalog}.{bronze_schema}.{pages_table_name}"
    print(f"\nUsing existing pages table: {pages_table_full}")

    base_pages_df = spark.table(pages_table_full)

    # Join in document_id / parent_document_id / file_extension from raw table
    raw_meta_df = (
        spark.table(raw_table_full)
        .select(
            "document_id",
            "parent_document_id",
            "file_name",
            "file_extension",
            "file_version",
        )
        .dropDuplicates(["parent_document_id", "file_name", "file_version"])
    )

    pages_df = base_pages_df.join(
        raw_meta_df,
        on=["file_name", "file_version"],
        how="left",
    )

else:
    print("\nNo pages_table_name configured or table not found – using raw table as single-page input.")
    raw_df = spark.table(raw_table_full)

    # Construct a single "page" per document using the full 'content'
    pages_df = raw_df.select(
        F.col("extracted_path").alias("file_path"),
        F.col("file_name"),
        F.col("file_version"),
        F.lit(0).alias("page_index"),
        F.col("content").alias("page_content"),
        F.col("document_id"),
        F.col("parent_document_id"),
        F.col("file_extension"),
    )

# Optional widget: restrict to a single file for testing
dbutils.widgets.text("file_name_filter", "", "Process only file_name (optional)")
file_name_filter = dbutils.widgets.get("file_name_filter").strip()

if file_name_filter:
    pages_df = pages_df.filter(F.col("file_name") == file_name_filter)

page_count = pages_df.count()
print(f"\nNumber of pages to process: {page_count}")

if page_count == 0:
    print("No pages to process. Exiting.")
    dbutils.notebook.exit("No OA pages found to process")


In [ ]:
# Cell 3: Model endpoint + prompt for section extraction

url = "https://imuj8rzcm7.execute-api.us-east-1.amazonaws.com/dev/genai-chat?api-version=2023-03-15-preview%22"

api_key = dbutils.secrets.get("dta_poc_api_key", "gpt_api_key")

headers = {
    "api-key": api_key,
    "Content-Type": "application/json",
}

MODEL_NAME = "gpt-4.1-mini-global"

prompt_header = """
You are analysing a contiguous fragment of a trial-specific data transfer agreement (tsDTA)
document. The input text may include several sections and subsections, headings, paragraphs
and sets of options (for example represented as checkboxes or structured choice lines).

Your task is to identify EVERY logical section or subsection in this text that presents one
or more options to choose from, and to output a structured JSON representation of those
sections.

You must capture:
- Top-level sections that introduce options.
- Nested subsections under larger sections whenever they introduce their own options.

For each such section or subsection you find, you must return an object with the fields:
- section_name
- section_description
- option_key
- options
- selected_options

WHAT COUNTS AS A SECTION OR SUBSECTION:
A "section" is any logical block of text that:
1) Has a heading or title (standalone title line, numbered heading, or short line introducing a topic), AND
2) Has one or more options directly associated with that heading or title.

Options typically appear as:
- Lines with checkbox-like markers (empty or filled squares, tick marks, bracketed markers), OR
- Clearly separated choice lines under an instruction to select or choose.

section_name (string): The heading, title, or local topic line that introduces the options.
section_description (string): Descriptive/instructional text between section_name and first option.
option_key (string): If section_description is non-empty, use it; otherwise use section_name.
options (array of strings): Labels for ALL options, with markers removed.
selected_options (array of strings): Labels that are clearly marked as selected (☒, ☑, ■, [X], [x]).

OUTPUT FORMAT:
- Your response MUST be valid JSON.
- The top-level structure MUST be a JSON array of zero or more objects.
- Each object MUST have exactly these fields: section_name, section_description, option_key, options, selected_options
- Do NOT include any text before or after the JSON array.

----- PAGE TEXT START -----
""".strip()

prompt_footer = "\n----- PAGE TEXT END -----"

print("✅ Model endpoint and prompt configured")


In [ ]:
# Cell 4: Section extraction function and UDF

def extract_sections_with_agent(page_text: str, debug: bool = False) -> str:
    """
    Call the GPT endpoint for a single page of text and return
    the raw JSON string (array of section objects).
    """
    full_prompt = f"{prompt_header}\n{page_text}\n{prompt_footer}"

    data = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "system",
                "content": "You are a precise information extraction assistant that outputs strict JSON only."
            },
            {
                "role": "user",
                "content": full_prompt
            },
        ],
    }

    resp = requests.post(url, headers=headers, json=data, timeout=120)
    resp_json = resp.json()

    if debug:
        print("=== STATUS CODE ===", resp.status_code)
        print("=== RESPONSE JSON (truncated) ===")
        print(json.dumps(resp_json, indent=2)[:4000])

    out = resp_json["choices"][0]["message"]["content"]
    return out.strip()


@F.udf(returnType=StringType())
def sections_json_udf(page_text: str) -> str:
    if page_text is None or page_text.strip() == "":
        return "[]"
    return extract_sections_with_agent(page_text)

print("✅ Section extraction UDF defined")


In [ ]:
# Cell 5: Call the model per page, parse JSON, and flatten sections

sections_json_df = pages_df.select(
    "file_path",
    "file_name",
    "file_version",
    "page_index",
    "document_id",
    "parent_document_id",
    "file_extension",
    sections_json_udf(F.col("page_content")).alias("sections_json"),
)

# Parse JSON and flatten sections
section_schema = ArrayType(
    StructType([
        StructField("section_name",        StringType(), True),
        StructField("section_description", StringType(), True),
        StructField("option_key",          StringType(), True),
        StructField("options",             ArrayType(StringType()), True),
        StructField("selected_options",    ArrayType(StringType()), True),
    ])
)

parsed_df = sections_json_df.select(
    "file_path",
    "file_name",
    "file_version",
    "page_index",
    "document_id",
    "parent_document_id",
    "file_extension",
    F.from_json("sections_json", section_schema).alias("sections"),
)

sections_exploded_df = parsed_df.select(
    "file_path",
    "file_name",
    "file_version",
    "page_index",
    "document_id",
    "parent_document_id",
    "file_extension",
    F.posexplode_outer("sections").alias("section_index", "section"),
)

sections_flat_df = sections_exploded_df.select(
    "file_path",
    "file_name",
    "file_version",
    "page_index",
    "document_id",
    "parent_document_id",
    "file_extension",
    "section_index",
    F.col("section.section_name").alias("section_name"),
    F.col("section.section_description").alias("section_description"),
    F.col("section.option_key").alias("option_key"),
    F.col("section.options").alias("options"),
    F.col("section.selected_options").alias("selected_options"),
)

sections_flat_df = sections_flat_df.where(F.col("section_name").isNotNull())

print(f"\nTotal section rows (before join): {sections_flat_df.count()}")


In [ ]:
# Cell 6: Enrich with OA header metadata from Silver

oa_header_table = f"{catalog}.{silver_schema}.md_dta_operational_agreement_draft"

if spark.catalog.tableExists(oa_header_table):
    oa_header_df = (
        spark.table(oa_header_table)
        .select(
            "operational_agreement_id",
            "parent_document_id",
            "trial_id",
            "data_stream",
            "data_provider_name",
            "dta_id",
            "version_tag",
            "version_status",
            "is_current_draft",
            "definition_hash",
            "row_status",
            "vendor_comments",
        )
        .dropDuplicates(["parent_document_id"])
    )

    sections_enriched_df = sections_flat_df.join(
        oa_header_df,
        on="parent_document_id",
        how="left",
    )
else:
    print(f"⚠ OA header table not found: {oa_header_table} – proceeding without header join.")
    sections_enriched_df = (
        sections_flat_df
        .withColumn("operational_agreement_id", F.lit(None).cast("string"))
        .withColumn("trial_id",                  F.lit(None).cast("string"))
        .withColumn("data_stream",               F.lit(None).cast("string"))
        .withColumn("data_provider_name",        F.lit(None).cast("string"))
        .withColumn("dta_id",                    F.lit(None).cast("string"))
        .withColumn("version_tag",               F.lit(None).cast("string"))
        .withColumn("version_status",            F.lit("DRAFT"))
        .withColumn("is_current_draft",          F.lit(True))
        .withColumn("definition_hash",
            oa_definition_hash_udf(
                F.col("operational_agreement_id"),
                F.col("trial_id"),
                F.col("data_stream"),
                F.col("data_provider_name"),
            )
        )
        .withColumn("row_status",        F.lit("COMPLETED"))
        .withColumn("vendor_comments",   F.lit(None).cast("string"))
    )

# Ensure definition_hash / row_status / vendor_comments are present
sections_enriched_df = (
    sections_enriched_df
    .withColumn(
        "definition_hash",
        F.when(F.col("definition_hash").isNotNull(),
               F.col("definition_hash"))
         .otherwise(
            oa_definition_hash_udf(
                F.col("operational_agreement_id"),
                F.col("trial_id"),
                F.col("data_stream"),
                F.col("data_provider_name"),
            )
        )
    )
    .withColumn(
        "row_status",
        F.when(F.col("row_status").isNotNull(), F.col("row_status"))
         .otherwise(F.lit("COMPLETED"))
    )
    .withColumn(
        "vendor_comments",
        F.when(F.col("vendor_comments").isNotNull(), F.col("vendor_comments"))
         .otherwise(F.lit(None).cast("string"))
    )
)

print(f"\nEnriched section rows: {sections_enriched_df.count()}")


In [ ]:
# Cell 7: Split into options vs other

# "Options" = all key section fields present and options array non-empty.
# "Other"   = everything else (missing/incomplete section fields).

options_condition = (
    F.col("section_name").isNotNull()
    & F.col("option_key").isNotNull()
    & F.col("options").isNotNull()
    & (F.size("options") > 0)
)

options_df = sections_enriched_df.where(options_condition)
other_df   = sections_enriched_df.where(~options_condition)

print(f"\nRows classified as options: {options_df.count()}")
print(f"Rows classified as other  : {other_df.count()}")


In [ ]:
# Cell 8: Build final schemas and write Silver tables with save_with_audit

def with_uuid(df, col_name):
    return df.withColumn(col_name, F.expr("uuid()"))

# ---- Options table ----

options_final_df = (
    with_uuid(options_df, "oa_options_id")
    .select(
        "oa_options_id",
        "operational_agreement_id",
        "document_id",
        "parent_document_id",
        "dta_id",
        "trial_id",
        "data_stream",
        "data_provider_name",
        "section_name",
        "section_description",
        "option_key",
        "options",
        "selected_options",
        "definition_hash",
        # status + row_status
        F.lit("COMPLETED").alias("status"),
        F.lit("COMPLETED").alias("row_status"),
        F.col("vendor_comments"),
        # file & versioning
        "file_name",
        "file_extension",
        "version_tag",
        "version_status",
        "is_current_draft",
    )
)

print("\nSample rows for md_dta_oa_options_draft:")
options_final_df.show(5, truncate=False)

save_with_audit(
    df=options_final_df,
    table_name=options_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode="overwrite",
    table_description=(
        "Normalised OA sections with options and selections extracted from "
        "operational agreement documents. One row per logical options section."
    ),
)

# ---- Other table ----

other_final_df = (
    with_uuid(other_df, "oa_other_id")
    .select(
        "oa_other_id",
        "operational_agreement_id",
        "document_id",
        "parent_document_id",
        "dta_id",
        "trial_id",
        "data_stream",
        "data_provider_name",
        "section_name",
        "section_description",
        "option_key",
        "options",
        "selected_options",
        "definition_hash",
        # status + row_status
        F.lit("MANUAL_REVIEW_REQUIRED").alias("status"),
        F.lit("MANUAL_REVIEW_REQUIRED").alias("row_status"),
        F.lit("Section incomplete or missing fields").alias("vendor_comments"),
        # file & versioning
        "file_name",
        "file_extension",
        "version_tag",
        "version_status",
        "is_current_draft",
    )
)

print("\nSample rows for md_dta_oa_other_draft:")
other_final_df.show(5, truncate=False)

save_with_audit(
    df=other_final_df,
    table_name=other_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode="overwrite",
    table_description=(
        "Operational agreement sections that could not be fully normalised. "
        "Intended for manual review and remediation."
    ),
)

print("\n✅ OA options/other normalisation completed")
print(f"   Options table: {options_table}")
print(f"   Other  table : {other_table}")

# Set task values for downstream notebooks
dbutils.jobs.taskValues.set(key="oa_options_status", value="SUCCESS")
dbutils.jobs.taskValues.set(key="oa_options_count", value=str(options_final_df.count()))
dbutils.jobs.taskValues.set(key="oa_other_count", value=str(other_final_df.count()))
